<h1><center>Black-Scholes Généralisé - Estimation de volatilité et pricing</center></h1>

Le but de ce notebook est de faire l'estimation de la volatilité implicite pour ensuite utiliser le modèle de Garman Kohlhagen (dérivé du modèle Black-Scholes généralisé) dans le but de faire le pricing des options FX.

In [1]:
from __future__ import division

# importation des librairies nécessaires
import unittest
import math
import numpy as np
from scipy.stats import norm
from scipy.stats import mvn

_DEBUG = False

In [2]:
# Cette classe contient les limites des entrées pour les modèles du GBS
# Ce n'est pas destiné à faire partie de l'interface de ce module
class _GBS_Limites:
    # Un modèle de GBS retournera une erreur si une entrée hors limite est saisie
    MAX32 = 2147483248.0

    MIN_T = 1.0 / 1000.0  # l'option exige qu'il reste un certain temps avant l'expiration
    MIN_X = 0.01
    MIN_FS = 0.01

    # Une volatilité inférieure à 0,5% entraîne l'échec des calculs d'options américaines

    MIN_V = 0.005

    MAX_T = 100
    MAX_X = MAX32
    MAX_FS = MAX32

    # Limites de l'option asiatique
    MIN_TA = 0

    # Ce modèle fonctionnera avec des valeurs plus élevées pour b, r et V.
    # Cependant, de telles valeurs sont extrêmement rares.  
    # Pour détecter certaines erreurs courantes, les taux d'intérêt et la volatilité sont plafonnés à 100 %. 
    
    MIN_b = -1
    MIN_r = -1

    MAX_b = 1
    MAX_r = 1
    MAX_V = 1

In [3]:
class GBS_CalculationError(Exception):
    def __init__(self, mismatch):
        Exception.__init__(self, mismatch)

In [4]:
# ------------------------------
# Cette fonction permet de vérifier que l'indicateur Call/Put est correctement saisi
def _test_option_type(option_type):
    if (option_type != "c") and (option_type != "p"):
        print("Entrée invalide option_type ({0}). Valeurs acceptables: c, p".format(option_type))

In [5]:
# ------------------------------
# Cette fonction permet de s'assurer que les entrées sont correctes
def _gbs_test_inputs(option_type, fs, x, t, r, b, v):
    # -----------
    # Les entrées de test sont raisonnables
    _test_option_type(option_type)

    if (x < _GBS_Limites.MIN_X) or (x > _GBS_Limites.MAX_X):
        print(
            "Prix de Strike invalide (X). Intervalle acceptable : {1} à {2}".format(x, _GBS_Limites.MIN_X,
                                                                                               _GBS_Limites.MAX_X))

    if (fs < _GBS_Limites.MIN_FS) or (fs > _GBS_Limites.MAX_FS):
        print(
            "Prix de Forward/Spot invalide (FS). Intervalle acceptable : {1} à {2}".format(fs,
                                                                                           _GBS_Limites.MIN_FS,
                                                                                           _GBS_Limites.MAX_FS))

    if (t < _GBS_Limites.MIN_T) or (t > _GBS_Limites.MAX_T):
        print(
            "Temps invalide (T = {0}). Intervalle acceptable : {1} à {2}".format(t, _GBS_Limites.MIN_T,
                                                                                             _GBS_Limites.MAX_T))

    if (b < _GBS_Limites.MIN_b) or (b > _GBS_Limites.MAX_b):
        print(
            "Cost of Carry invalide (b = {0}). Intervalle acceptable : {1} à {2}".format(b,
                                                                                        _GBS_Limites.MIN_b,
                                                                                        _GBS_Limites.MAX_b))

    if (r < _GBS_Limites.MIN_r) or (r > _GBS_Limites.MAX_r):
        print(
            "Taux actif sans risque invalide (r = {0}). Intervalle acceptable : {1} à {2}".format(r,
                                                                                                 _GBS_Limites.MIN_r,
                                                                                                 _GBS_Limites.MAX_r))

    if (v < _GBS_Limites.MIN_V) or (v > _GBS_Limites.MAX_V):
        print(
            "Volatilité implicite invalide (V = {0}). Intervalle acceptable : {1} à {2}".format(v,
                                                                                                _GBS_Limites.MIN_V,
                                                                                                _GBS_Limites.MAX_V))

In [6]:
# Entrées: type_option = "p" or "c", fs = prix de l'actif sous-jacent, x = strike, t = échéance, r = taux actif sans risque
#         b = terme de correction, v = volatilité implicite
# Sorties: prix, delta, gamma, theta, vega, rho
def _gbs(option_type, fs, x, t, r, b, v):
    
    _gbs_test_inputs(option_type, fs, x, t, r, b, v)
    # Calculs préliminaires
    t__sqrt = math.sqrt(t)
    d1 = (math.log(fs / x) + (b + (v * v) / 2) * t) / (v * t__sqrt)
    d2 = d1 - v * t__sqrt

    if option_type == "c":
        # Call
        value = fs * math.exp((b - r) * t) * norm.cdf(d1) - x * math.exp(-r * t) * norm.cdf(d2)
        delta = math.exp((b - r) * t) * norm.cdf(d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) - (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(d1) - r * x * math.exp(-r * t) * norm.cdf(d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = x * t * math.exp(-r * t) * norm.cdf(d2)
    else:
        # Put
        value = x * math.exp(-r * t) * norm.cdf(-d2) - (fs * math.exp((b - r) * t) * norm.cdf(-d1))
        delta = -math.exp((b - r) * t) * norm.cdf(-d1)
        gamma = math.exp((b - r) * t) * norm.pdf(d1) / (fs * v * t__sqrt)
        theta = -(fs * v * math.exp((b - r) * t) * norm.pdf(d1)) / (2 * t__sqrt) + (b - r) * fs * math.exp(
            (b - r) * t) * norm.cdf(-d1) + r * x * math.exp(-r * t) * norm.cdf(-d2)
        vega = math.exp((b - r) * t) * fs * t__sqrt * norm.pdf(d1)
        rho = -x * t * math.exp(-r * t) * norm.cdf(-d2)
        
    return value, delta, gamma, theta, vega, rho  

 ## Volatilité implicite

In [7]:
# ----------
# Entrées
#      fs = Prix forward/spot
#      x = Strike
#      t = Temps (en années)
#      r = taux actif sans risque
#      b = coût
#      cp = prix du call/put
#      precision = (optionnelle)  
#      max_steps = (optionnelle) 

# ---------
# Cette fonction permet de choisir un point de départ pour 
# les fonctions de recherche (recherche Newton et dichotomique). 

def _vol_implicite_approx(option_type, fs, x, t, r, b, cp):
    _test_option_type(option_type)

    ebrt = math.exp((b - r) * t)
    ert = math.exp(-r * t)

    a = math.sqrt(2 * math.pi) / (fs * ebrt + x * ert)

    if option_type == "c":
        payoff = fs * ebrt - x * ert
    else:
        payoff = x * ert - fs * ebrt

    b = cp - payoff / 2
    c = (payoff ** 2) / math.pi

    v = (a * (b + math.sqrt(b ** 2 + c))) / math.sqrt(t)

    return v

In [8]:
# ----------
# Trouver la volatilité implicite d'une option européenne (GBS) à un prix donné en utilisant la méthode Newton-Raphson 
# pour plus de rapidité puisque Vega est disponible

def _gbs_vol_implicite(option_type, fs, x, t, r, b, cp, precision=.00001, max_steps=100):
    return _newton_vol_implicite(_gbs, option_type, x, fs, t, b, r, cp, precision, max_steps)

In [9]:
# ----------
# Calculer la volatilité implicite avec une recherche Newton Raphson
def _newton_vol_implicite(val_fn, option_type, x, fs, t, b, r, cp, precision=.00001, max_steps=100):
    # verifier le type d'option
    _test_option_type(option_type)

    # Vérification de l'appartenance au domaine de def
    v = _vol_implicite_approx(option_type, fs, x, t, r, b, cp)
    v = max(_GBS_Limites.MIN_V, v)
    v = min(_GBS_Limites.MAX_V, v)

    value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
    min_diff = abs(cp - value)

    # Recherche Newton-Raphson
    countr = 0
    while precision <= abs(cp - value) <= min_diff and countr < max_steps:

        v = v - (value - cp) / vega
        if (v > _GBS_Limites.MAX_V) or (v < _GBS_Limites.MIN_V):
            print("    Volatilité hors limites")
            break

        value, delta, gamma, theta, vega, rho = val_fn(option_type, fs, x, t, r, b, v)
        min_diff = min(abs(cp - value), min_diff)

        countr += 1
        #print("     VOL_IMP ITER {0}. v={1}".format(countr, v))
 
    # vérifier la convergence et retourner la valeur
    if abs(cp - value) < precision:
        # la fct de recherche a convergé
        return v
    else:
        # si la recherche ne converge pas, essayer la recherche dichotomique
        return _rech_dichotomique_vol_implicite(val_fn, option_type, fs, x, t, r, b, cp, precision, max_steps)

In [10]:
# trouver volatilité implicite en utilisant la recherche dichotomique
def _rech_dichotomique_vol_implicite(val_fn, option_type, fs, x, t, r, b, cp, precision=.00001, max_steps=100):

    v_mid = _vol_implicite_approx(option_type, fs, x, t, r, b, cp)

    if (v_mid <= _GBS_Limites.MIN_V) or (v_mid >= _GBS_Limites.MAX_V):
        # si l'estimation de la volatilité est hors limites, rechercher dans tout l'espace de vol. autorisé
        v_low = _GBS_Limites.MIN_V
        v_high = _GBS_Limites.MAX_V
        v_mid = (v_low + v_high) / 2
    else:
        # réduire la taille de l'espace de vol.
        v_low = max(_GBS_Limites.MIN_V, v_mid * .5)
        v_high = min(_GBS_Limites.MAX_V, v_mid * 1.5)

    # Estimer les limites haute/basse du prix
    cp_mid = val_fn(option_type, fs, x, t, r, b, v_mid)[0]
    
    # initialiser la recherche dichotomique
    current_step = 0
    diff = abs(cp - cp_mid)

    # Maintenir la volatilité dichotomique jusqu'à ce qu'un prix correct soit trouvé
    while (diff > precision) and (current_step < max_steps):
        current_step += 1

        # Diviser la zone de recherche en deux
        if cp_mid < cp:
            v_low = v_mid
        else:
            v_high = v_mid

        cp_low = val_fn(option_type, fs, x, t, r, b, v_low)[0]
        cp_high = val_fn(option_type, fs, x, t, r, b, v_high)[0]

        v_mid = v_low + (cp - cp_low) * (v_high - v_low) / (cp_high - cp_low)
        v_mid = max(_GBS_Limites.MIN_V, v_mid)  
        v_mid = min(_GBS_Limites.MAX_V, v_mid)  

        cp_mid = val_fn(option_type, fs, x, t, r, b, v_mid)[0]
        diff = abs(cp - cp_mid)

        #print("     VOL_IMP {0}. V[{1},{2},{3}]".format(current_step, v_low, v_mid, v_high))

    # retourner l'output
    if abs(cp - cp_mid) < precision:
        return v_mid
    else:
        raise GBS_CalculationError(
            "Vol implicite n_a pas convergé. Meilleure estimation={0}, Diff prix={1}, Precision requise={2}".format(v_mid, diff,
                                                                                                          precision))

*Entrées:*
<br>
   * option_type = "p" ou "c"
   * fs          = prix du sous-jacent
   * x           = strike
   * t           = échéance
   * v           = volatilité implicite
   * r           = taux sans risque
   * q           = paiement de dividendes
   * b           = coût de portage


*Sorties:*
<br>
   * value       = prix de l'option
   * delta       = première dérivée du prix de l'option par rapport au prix du sous-jacent
   * gamma       = seconde dérivée du prix de l'option par rapport au prix du sous-jacent
   * theta       = première dérivée du prix de l'option par rapport au délai d'expiration
   * vega        = première dérivée du prix de l'option par rapport à la volatilité implicite
   * rho         = première dérivée du prix de l'option par rapport au taux sans risque

In [11]:
# ---------------------------
# Black Scholes: les options sur actions
def black_scholes(option_type, fs, x, t, r, v):
    b = r
    bs = _gbs(option_type, fs, x, t, r, b, v)
    print("Prix :  {0}\nDelta :  {1}\nGamma :  {2}\nTheta :  {3}\nVega :  {4}\nRho :  {5}\n".format(bs[0],bs[1],bs[2],
                                                                                                    bs[3],bs[4],bs[5]))
    return _gbs(option_type, fs, x, t, r, b, v)

In [12]:
# ---------------------------
# Options FX
def garman_kohlhagen(option_type, fs, x, t, r, rf, v):
    b = r - rf
    gk = _gbs(option_type, fs, x, t, r, b, v)
    print("Prix :  {0}\nDelta :  {1}\nGamma :  {2}\nTheta :  {3}\nVega :  {4}\nRho :  {5}\n".format(gk[0],gk[1],gk[2],
                                                                                                    gk[3],gk[4],gk[5]))
    return gk 

In [13]:
def euro_vol_implicite(option_type, fs, x, t, r, q, cp):
    b = r - q
    return _gbs_vol_implicite(option_type, fs, x, t, r, b, cp)

#### On effectue des tests pour vérifier 

In [14]:
euro_vol_implicite("c",fs=100,x=100,t=.5,r=.03,q=.01,cp=11.10)

0.38059506228385803

In [15]:
euro_vol_implicite("p",fs=100,x=100,t=.5,r=.03,q=.01,cp=11.10)

0.4164207070294432

In [16]:
### à verif :
black_scholes('p', fs=197.11, x=210, t=38/365, r=.01, v=0.5)

Prix :  20.376582927215637
Delta :  -0.6200178074674287
Gamma :  0.011973358188266586
Theta :  -56.72325808917273
Vega :  24.215532698216656
Rho :  -14.844808581837206



(20.376582927215637,
 -0.6200178074674287,
 0.011973358188266586,
 -56.72325808917273,
 24.215532698216656,
 -14.844808581837206)

In [17]:
garman_kohlhagen('c', fs=1.56, x=1.60, t=0.5, r=0.06, rf=0.08, v=0.12)

Prix :  0.02909925314943973
Delta :  0.3403859092321427
Gamma :  2.7002660835461674
Theta :  -0.03494785073760004
Vega :  0.3942820524550773
Rho :  0.25095138262635147



(0.02909925314943973,
 0.3403859092321427,
 2.7002660835461674,
 -0.03494785073760004,
 0.3942820524550773,
 0.25095138262635147)

In [18]:
x = garman_kohlhagen('p', fs=1.56, x=1.60, t=0.5, r=0.06, rf=0.08, v=0.12)

Prix :  0.08298058174942846
Delta :  -0.6204035299201804
Gamma :  2.7002660835461674
Theta :  -0.06169160152315317
Vega :  0.3942820524550773
Rho :  -0.525405044212455



In [19]:
garman_kohlhagen('c', fs=0.7, x=0.72, t=0.5, r=0.0575, rf=0.0625, v=0.2756)

Prix :  0.043577282747810364
Delta :  0.4614515969182755
Gamma :  2.829409486282998
Theta :  -0.04853188560170679
Vega :  0.1910473873328006
Rho :  0.13971941754749123



(0.043577282747810364,
 0.4614515969182755,
 2.829409486282998,
 -0.04853188560170679,
 0.1910473873328006,
 0.13971941754749123)

In [20]:
garman_kohlhagen('c', fs=0.7, x=.72, t=.5, r=0.0575, rf=0.0625, v=0.2756)

Prix :  0.043577282747810364
Delta :  0.4614515969182755
Gamma :  2.829409486282998
Theta :  -0.04853188560170679
Vega :  0.1910473873328006
Rho :  0.13971941754749123



(0.043577282747810364,
 0.4614515969182755,
 2.829409486282998,
 -0.04853188560170679,
 0.1910473873328006,
 0.13971941754749123)

In [21]:
garman_kohlhagen('p', fs=0.7, x=.72, t=.5, r=0.0575, rf=0.0625, v=0.2756)

Prix :  0.06470874985262098
Delta :  -0.5077816375580687
Gamma :  2.829409486282998
Theta :  -0.050709142563847384
Vega :  0.1910473873328006
Rho :  -0.21007794807163452



(0.06470874985262098,
 -0.5077816375580687,
 2.829409486282998,
 -0.050709142563847384,
 0.1910473873328006,
 -0.21007794807163452)

In [22]:
garman_kohlhagen('c', fs=1, x=1.15, t=0.474, r=1, rf=0.84, v=0.068)

Prix :  0.0012837604379233175
Delta :  0.06030789379973272
Gamma :  2.3256181169089385
Theta :  -0.013742331656327392
Vega :  0.0749593231442089
Rho :  0.027977439213497657



(0.0012837604379233175,
 0.06030789379973272,
 2.3256181169089385,
 -0.013742331656327392,
 0.0749593231442089,
 0.027977439213497657)

In [34]:
black_scholes('c', 20, 18, 1, .05, .15)

Prix :  3.093431812651085
Delta :  0.8666591651857289
Gamma :  0.07176065675591503
Theta :  -1.0349105299547925
Vega :  4.305639405354902
Rho :  14.239751491063492



(3.093431812651085,
 0.8666591651857289,
 0.07176065675591503,
 -1.0349105299547925,
 4.305639405354902,
 14.239751491063492)

In [35]:
black_scholes('p', 20, 18, 1, .05, .15)

Prix :  0.21556145366393764
Delta :  -0.13334083481427106
Gamma :  0.07176065675591503
Theta :  -0.1788040479041497
Vega :  4.305639405354902
Rho :  -2.882378149949359



(0.21556145366393764,
 -0.13334083481427106,
 0.07176065675591503,
 -0.1788040479041497,
 4.305639405354902,
 -2.882378149949359)

In [33]:
euro_vol_implicite("p",fs=100,x=100,t=1,r=.02,q=0,cp=10)

0.27842040206886837